# 7章 ロジスティクスネットワークの最適設計を行う10本ノック

ここでは、最適化計算を行ういくつかのライブラリを用いて、最適化計算を実際に行っていきます。  
そして、前章で用いたネットワーク可視化などの技術を駆使し、計算結果の妥当性を確認する方法についても学んでいきます。

### ノック６１：輸送最適化問題を解いてみよう

In [4]:
import numpy as np
import pandas as pd
from itertools import product
from pulp import LpVariable, lpSum, value
from ortoolpy import model_min, addvars, addvals

# データ読み込み
df_tc = pd.read_csv("trans_cost.csv", index_col="工場")
df_demand = pd.read_csv("demand.csv")
df_supply = pd.read_csv("supply.csv")



In [6]:
df_tc

,F1,F2,F3,F4
工場,,,,
W1,10,10,11,27
W2,18,21,12,14
W3,15,12,14,12


In [7]:
df_demand

,F1,F2,F3,F4
0,28,29,31,25


In [8]:
df_supply

,W1,W2,W3
0,35,41,42


In [10]:
# 初期設定
np.random.seed(1)
nw = len(df_tc.index)
nf = len(df_tc.columns)
pr = list(product(range(nw), range(nf)))

In [11]:
pr

[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (1, 0),
 (1, 1),
 (1, 2),
 (1, 3),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 3)]

In [14]:
# 数理モデル作成
m1 = model_min()
v1 = {(i,j):LpVariable("v%d_%d"%(i,j), lowBound=0) for i,j in pr}

m1 += lpSum(df_tc.iloc[i][j]*v1[i,j] for i,j in pr)
for i in range(nw):
    m1 += lpSum(v1[i,j] for j in range(nf)) <= df_supply.iloc[0][i]
for j in range(nf):
    m1 += lpSum(v1[i,j] for j in range(nw)) >= df_demand.iloc[0][i]
m1.solve()   

1

In [16]:
# 総輸送コスト計算
df_tr_sol = df_tc.copy()
total_cost = 0
for k , x in v1.items():
    i,j = k[0],k[1]
    df_tr_sol.iloc[i][j] = value(x)
    total_cost += df_tc.iloc[i][j]*value(x)
    
print(df_tr_sol)
print("総輸送コスト:" + str(total_cost))

    F1  F2  F3  F4
工場                
W1   0   0   0   0
W2   0   0   0   0
W3   0  31   0   0
総輸送コスト:372.0


### ノック６２：最適輸送ルートをネットワークで確認しよう

### ノック６３：最適輸送ルートが制約条件内に収まっているかどうかを確認しよう

### ノック６４：生産計画に関するデータを読み込んでみよう

### ノック６５：利益を計算する関数を作ってみよう

### ノック６６：生産最適化問題を解いてみよう

### ノック６７：最適生産計画が制約条件内に収まっているかどうかを確認しよう

### ノック６８：ロジスティクスネットワーク設計問題を解いてみよう

### ノック６９：最適ネットワークにおける輸送コストとその内訳を計算しよう

### ノック７０：最適ネットワークにおける生産コストとその内訳を計算しよう